In [38]:
!pip install pandas "google-generativeai" python-dotenv jupyter kaggle

In [39]:
!pip install nbformat

In [40]:
# kaggle competitions download -c nlp-getting-started -p data
# Above to be run in terminal to download data from Kaggle

In [41]:
import pandas as pd
import os
from dotenv import load_dotenv
import google.generativeai as genai

In [42]:
# project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# dotenv_path = os.path.join(project_root, '.env')

# found_dotenv = load_dotenv(dotenv_path=dotenv_path, override=True)

# if not found_dotenv:
#     print(f"Warning: Could not find .env file at path: {dotenv_path}")

load_dotenv()

True

In [43]:
data_path = r'Datasets/NLP with Disaster Tweets'

In [15]:
df = pd.read_csv(f'{data_path}/train.csv')
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [44]:
GOOGLE_API_KEY = os.getenv('GEMINI_API_KEY')
# if not GOOGLE_API_KEY:
#     # If the key is not found, raise an error with a helpful message.
#     raise ValueError("API Key not found. Please ensure your .env file is in the project root and contains GOOGLE_API_KEY.")
# else:
#     print("Successfully loaded API Key.")

# try:
#     # Configure the genai library with your key
#     genai.configure(api_key=GOOGLE_API_KEY)
#     print("Gemini API configured successfully!")

#     # You can now proceed to use the model
#     # model = genai.GenerativeModel('gemini-pro')
#     # ... your code here ...

# except Exception as e:
#     # Catch potential configuration errors
#     print(f"An error occurred during API configuration: {e}")

genai.configure(api_key=GOOGLE_API_KEY)

In [27]:
model = genai.GenerativeModel('gemini-2.5-flash')

In [61]:
# --- Let's test it on a sample disaster tweet (row 31) ---
sample_tweet = df.iloc[31]
sample_tweet

id                                                         48
keyword                                                ablaze
location                                           Birmingham
text        @bbcmtd Wholesale Markets ablaze http://t.co/l...
target                                                      1
Name: 31, dtype: object

In [63]:
row_data_as_string = "\n".join([f"{col_name}: {value}" for col_name, value in sample_tweet.items()])
json_prompt = f"""
You are a disaster analysis system. Analyze the following social media post.
Your task is to extract key information and return it ONLY as a valid JSON object.

The JSON object should have the following fields:
- "is_disaster": boolean (true if the post is about a real disaster, otherwise false)
- "disaster_type": string (e.g., "Fire", "Flood", "Earthquake", "Bombing", "None")
- "location_mentioned": string (the specific location mentioned, or "Unknown")
- "severity_score": integer (a score from 1-5, where 5 is most severe; 0 if not a disaster)

AS input you have a list of the python dataframe row with attributes as:
- id - a unique identifier for each tweet
- text - the text of the tweet
- location - the location the tweet was sent from (may be blank)
- keyword - a particular keyword from the tweet (may be blank)
- target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

Also don't forget to consider the context of the tweet, as some tweets may use figurative language or sarcasm.
e.g., "I am on fire today!" is not about a real fire disaster.
e.g.  "@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C" is just about prices in the market too high leading to use of metaphorical language = "Wholesale Markets ablaze"
Also if you know various Twitter handles, hashtags or keywords that are associated with disasters, use that knowledge to inform your analysis.

Post: "{sample_tweet}"
"""

In [64]:
response = model.generate_content(json_prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n  \"is_disaster\": false,\n  \"disaster_type\": \"None\",\n  \"location_mentioned\": \"Unknown\",\n  \"severity_score\": 0\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 415,
        "candidates_token_count": 48,
        "total_token_count": 1128
      },
      "model_version": "gemini-2.5-flash"
    }),
)

In [65]:
print(f"TWEET: '{sample_tweet['text']}'")
print("---------------------------------------------")
print("MODEL'S STRUCTURED JSON OUTPUT:")
print(response.text)

TWEET: '@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C'
---------------------------------------------
MODEL'S STRUCTURED JSON OUTPUT:
```json
{
  "is_disaster": false,
  "disaster_type": "None",
  "location_mentioned": "Unknown",
  "severity_score": 0
}
```


In [67]:
sample_tweet = df.iloc[100]
sample_tweet

id                                                        144
keyword                                              accident
location                                                   UK
text        .@NorwayMFA #Bahrain police had previously die...
target                                                      1
Name: 100, dtype: object

In [ ]:
row_data_as_string = "\n".join([f"{col_name}: {value}" for col_name, value in sample_tweet.items()])
json_prompt = f"""
You are a disaster analysis system. Analyze the following social media post.
Your task is to extract key information and return it ONLY as a group of valid JSON objects for each row a separate JSON object as described below so that I can process your given JSON directly to save the results into a csv.

The JSON object should have the following fields:
- "is_disaster": boolean (true if the post is about a real disaster, otherwise false)
- "disaster_type": string (e.g., "Fire", "Flood", "Earthquake", "Bombing", "None")
- "disaster_severity": string (e.g., "Severe", "Moderate", "Mild", "None")
- "Resources/Help Type needed": string (e.g., "Medical", "Food", "Shelter", "None",) #This should be as per your reasoning understanding context of the statement and what type of help would be required in that situation if that was happening right in front of you.
- "location_mentioned": string (the specific location mentioned, or "Unknown")
- "severity_score": integer (a score from 1-5, where 5 is most severe; 0 if not a disaster)

AS input you have a list of the python dataframe row with attributes as:
- id - a unique identifier for each tweet
- text - the text of the tweet
- location - the location the tweet was sent from (may be blank)
- keyword - a particular keyword from the tweet (may be blank)
- target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

Also don't forget to consider the context of the tweet, as some tweets may use figurative language or sarcasm.
e.g., "I am on fire today!" is not about a real fire disaster.
e.g.  "@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C" is just about prices in the market too high leading to use of metaphorical language = "Wholesale Markets ablaze"
Also if you know various Twitter handles, hashtags or keywords that are associated with disasters, use that knowledge to inform your analysis.

Post: "{sample_tweet}"
"""

In [69]:
response = model.generate_content(json_prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n  \"is_disaster\": true,\n  \"disaster_type\": \"Accident\",\n  \"location_mentioned\": \"Bahrain\",\n  \"severity_score\": 4\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 413,
        "candidates_token_count": 50,
        "total_token_count": 1086
      },
      "model_version": "gemini-2.5-flash"
    }),
)

In [70]:
print(f"TWEET: '{sample_tweet['text']}'")
print("---------------------------------------------")
print("MODEL'S STRUCTURED JSON OUTPUT:")
print(response.text)

TWEET: '.@NorwayMFA #Bahrain police had previously died in a road accident they were not killed by explosion https://t.co/gFJfgTodad'
---------------------------------------------
MODEL'S STRUCTURED JSON OUTPUT:
```json
{
  "is_disaster": true,
  "disaster_type": "Accident",
  "location_mentioned": "Bahrain",
  "severity_score": 4
}
```


In [45]:
# Now lets complete process the full testing dataset and save the results to a new CSV file.
test_df = pd.read_csv(f'{data_path}/test.csv')
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [46]:
model = genai.GenerativeModel('gemini-2.0-flash')

In [30]:
sample_tweets = test_df

In [ ]:
row_data_as_string = "\n".join([f"{col_name}: {value}" for col_name, value in sample_tweet.items()])
json_prompt = f"""
You are a disaster analysis system. Analyze the following social media post.
Your task is to extract key information and return it ONLY as a valid JSON object.

The JSON object should have the following fields:
- "is_disaster": boolean (true if the post is about a real disaster, otherwise false)
- "disaster_type": string (e.g., "Fire", "Flood", "Earthquake", "Bombing", "None")
- "disaster_severity": string (e.g., "Severe", "Moderate", "Mild", "None")
- "Resources/Help Type needed": string (e.g., "Medical", "Food", "Shelter", "None",) #This should be as per your reasoning understanding context of the statement and what type of help would be required in that situation if that was happening right in front of you.
- "location_mentioned": string (the specific location mentioned, or "Unknown")
- "severity_score": integer (a score from 1-5, where 5 is most severe; 0 if not a disaster)

AS input the python dataframe with 3263 rows with attributes as:
- id - a unique identifier for each tweet
- text - the text of the tweet
- location - the location the tweet was sent from (may be blank)
- keyword - a particular keyword from the tweet (may be blank)
- target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

Also don't forget to consider the context of the tweet, as some tweets may use figurative language or sarcasm.
e.g., "I am on fire today!" is not about a real fire disaster.
e.g.  "@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C" is just about prices in the market too high leading to use of metaphorical language = "Wholesale Markets ablaze"
Also if you know various Twitter handles, hashtags or keywords that are associated with disasters, use that knowledge to inform your analysis.

Post: "{sample_tweets}"
"""

In [47]:
def process_tweets_in_batch(tweets_df: pd.DataFrame) -> list[dict]:
    """
    Processes a batch of tweets using a single Gemini API call.
    """
    # 1. Format each row into a clearly separated string block.
    # We use the robust explicit formatting for each tweet.
    formatted_tweets = []
    for index, row in tweets_df.iterrows():
        tweet_data = "\n".join([f"{col}: {val}" for col, val in row.items()])
        # Add an identifier to each block
        formatted_tweets.append(f"--- TWEET (id: {row['id']}) ---\n{tweet_data}")

    # 2. Join all formatted tweet blocks into one large string.
    batch_string = "\n\n".join(formatted_tweets)

    # 3. Create a prompt that instructs the model to handle a batch
    #    and return a JSON array.
    json_prompt = f"""
    You are a disaster analysis system. Analyze the following social media post.
    Your task is to extract key information and return it ONLY as a valid JSON object.

    The JSON object should have the following fields:
    - "id": integer (the unique identifier for each tweet to help map results back) mandatory field, it should be the same id as you get with the inputs for each row
    - "is_disaster": boolean (true if the post is about a real disaster, otherwise false)
    - "disaster_type": string (e.g., "Fire", "Flood", "Earthquake", "Bombing", "None")
    - "disaster_severity": string (e.g., "Severe", "Moderate", "Mild", "None")
    - "Resources/Help Type needed": string (e.g., "Medical", "Food", "Shelter", "None",) #This should be as per your reasoning understanding context of the statement and what type of help would be required in that situation if that was happening right in front of you.
    - "location_mentioned": string (the specific location mentioned, or "Unknown") # If both location column has location and text column has location use have to integrate both to give best possible answer
    - "severity_score": integer (a score from 1-5, where 5 is most severe; 0 if not a disaster)


    AS input the python dataframe with 3263 rows with attributes as:
    - id - a unique identifier for each tweet
    - text - the text of the tweet
    - location - the location the tweet was sent from (may be blank)
    - keyword - a particular keyword from the tweet (may be blank)
    - target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)

    Also don't forget to consider the context of the tweet, as some tweets may use figurative language or sarcasm.
    e.g., "I am on fire today!" is not about a real fire disaster.
    e.g.  "@bbcmtd Wholesale Markets ablaze http://t.co/lHYXEOHY6C" is just about prices in the market too high leading to use of metaphorical language = "Wholesale Markets ablaze"
    Also if you know various Twitter handles, hashtags or keywords that are associated with disasters, use that knowledge to inform your analysis.

    Remember you have to give back all 3263 results so process deeply and carefully.

    Here is the batch of tweets to process:
        --- BATCH START ---
        {batch_string}
        --- BATCH END ---
    """
    return model.generate_content(json_prompt)
    

In [82]:
batch_results = process_tweets_in_batch(sample_tweets)
batch_results

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n[\n  {\n    \"is_disaster\": true,\n    \"disaster_type\": \"Car Crash\",\n    \"disaster_severity\": \"Terrible\",\n    \"Resources/Help Type needed\": \"Medical\",\n    \"location_mentioned\": \"Unknown\",\n    \"severity_score\": 3\n  },\n  {\n    \"is_disaster\": true,\n    \"disaster_type\": \"Earthquake\",\n    \"disaster_severity\": \"Unknown\",\n    \"Resources/Help Type needed\": \"None\",\n    \"location_mentioned\": \"cities\",\n    \"severity_score\": 2\n  },\n  {\n    \"is_disaster\": true,\n    \"disaster_type\": \"Fire\",\n    \"disaster_severity\": \"Unknown\",\n    \"Resources/Help Type needed\": \"None\",\n    \"location_mentioned\": \"spot pond\",\n    \"severity_score\": 3\n  },\n  {\n    \"is_disaster\": true,\n    \"disaster_ty

In [85]:
batch_results.text

'```json\n[\n  {\n    "is_disaster": true,\n    "disaster_type": "Car Crash",\n    "disaster_severity": "Terrible",\n    "Resources/Help Type needed": "Medical",\n    "location_mentioned": "Unknown",\n    "severity_score": 3\n  },\n  {\n    "is_disaster": true,\n    "disaster_type": "Earthquake",\n    "disaster_severity": "Unknown",\n    "Resources/Help Type needed": "None",\n    "location_mentioned": "cities",\n    "severity_score": 2\n  },\n  {\n    "is_disaster": true,\n    "disaster_type": "Fire",\n    "disaster_severity": "Unknown",\n    "Resources/Help Type needed": "None",\n    "location_mentioned": "spot pond",\n    "severity_score": 3\n  },\n  {\n    "is_disaster": true,\n    "disaster_type": "Wildfire",\n    "disaster_severity": "Unknown",\n    "Resources/Help Type needed": "None",\n    "location_mentioned": "Spokane",\n    "severity_score": 3\n  },\n  {\n    "is_disaster": true,\n    "disaster_type": "Typhoon",\n    "disaster_severity": "Severe",\n    "Resources/Help Type ne

In [ ]:
# Since we have large datasets we have to send data in small packets like in batches of 100 rows with a cooling period of 60 seconds between each batch to avoid rate limiting by the API.
# The last batch may have less than 100 rows and even in a batch the last part can be truncated which we need to handle in our code.
# We will also implement error handling to retry failed batches.
import pandas as pd
import json
import re
import time
def send_tweets_in_batches(sample_tweets_batch: pd.DataFrame) -> str:
    batch_results = process_tweets_in_batch(sample_tweets_batch)
    return batch_results
def process_received_text_toDataFrame(batch_results: str):
    ##Adding code to skip last truncated part of the batch if any
    # --- 2. Extract the JSON String from the Model's Response ---
    # We use a regex to find the JSON code block in the response.

    # below first we will extract the json part from the response text but if last part of the batch is truncated the below statement can fail
    # json_string = re.search(r"\[.*\]", batch_results, re.DOTALL).group(0)

    # So handling and avoiding that fail part
    # But we need other json objects which are not truncated so we will use try except block
    try:
        json_string = re.search(r"\[.*\]", batch_results, re.DOTALL).group(0)
    except AttributeError:
        print("Warning: The response does not contain a complete JSON array. Attempting to extract JSON code block instead.")
    
    # Try to extract JSON code block instead
    json_match = re.search(r"```json\n(.*?)\n```", batch_results, re.DOTALL)

    # Initialize an empty DataFrame to handle cases where parsing fails
    results_df = pd.DataFrame()

    if json_match:
        json_string = json_match.group(1)
        print("--- Successfully Extracted JSON String ---")
        print(json_string)
        print("-" * 40)

        # --- 3. Parse the JSON String and Convert to DataFrame ---
        try:
            # The json.loads() function parses the string into a Python list of dictionaries
            data = json.loads(json_string)
        
            # pandas can create a DataFrame directly from a list of dictionaries
            results_df = pd.DataFrame(data)
            print(results_df.head(3))
        
            print("\n--- Successfully Converted to DataFrame ---")

        except json.JSONDecodeError as e:
            print(f"Error: Failed to decode JSON. The extracted string may be malformed. Details: {e}")
    else:
        print("Error: Could not find a JSON code block in the LLM response.")
        return "JSON Not Found"
    return results_df



In [57]:
batch_results = {}
final_results_df = pd.DataFrame()
batch_size = 100
num_batches = (len(test_df) + batch_size - 1) // batch_size

In [61]:
#Send tweets of 200 rows each to send_tweets_in_batches()
#At the same moment start timer  and start doing other tasks if other tasks take more than 60 seconds then good send another batch otherwise wait for 60 seconds and then send another batch otherwise wait for remaining out of 60 seconds and then send another batch
# Also we will store previous responses so that we can start from where we left if by chance get stopped in between a response
for i in range(num_batches):
    start_time = time.time()
    
    batch_start = i * batch_size
    batch_end = min((i + 1) * batch_size, len(test_df))
    sample_tweets_batch = test_df.iloc[batch_start:batch_end]
    
    print(f"\n--- Processing Batch {i+1}/{num_batches} (Rows {batch_start} to {batch_end-1}) ---")
    
    batch_results[i] = send_tweets_in_batches(sample_tweets_batch)
    batch_df = process_received_text_toDataFrame(batch_results[i].text)
    #Need to first check if batch_df is not a dataframe then check for string
    if (type(batch_df) == str) and (batch_df == "JSON Not Found"):
        print(f"Batch {i+1} still failed to produce valid JSON. Skipping this batch.")
    
    elif not batch_df.empty:
        final_results_df = pd.concat([final_results_df, batch_df], ignore_index=True)
    
    # Calculate elapsed time and determine if we need to wait
    elapsed_time = time.time() - start_time
    if elapsed_time < 60:
        time_to_wait = 60 - elapsed_time
        print(f"Waiting for {time_to_wait:.2f} seconds to avoid rate limiting...")
        time.sleep(time_to_wait)
    else:
        print("No need to wait, proceeding to next batch immediately.")
final_results_df


--- Processing Batch 1/33 (Rows 0 to 99) ---
--- Successfully Extracted JSON String ---
[
  {
    "id": 0,
    "is_disaster": true,
    "disaster_type": "Car Crash",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "Medical",
    "location_mentioned": "Unknown",
    "severity_score": 3
  },
  {
    "id": 2,
    "is_disaster": true,
    "disaster_type": "Earthquake",
    "disaster_severity": "Mild",
    "Resources/Help Type needed": "None",
    "location_mentioned": "Unknown",
    "severity_score": 2
  },
  {
    "id": 3,
    "is_disaster": true,
    "disaster_type": "Fire",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "Animal Rescue",
    "location_mentioned": "spot pond",
    "severity_score": 3
  },
  {
    "id": 9,
    "is_disaster": true,
    "disaster_type": "Wildfire",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "None",
    "location_mentioned": "Spokane",
    "severity_score": 3
  },
  {
    "id": 11,
  

,id,is_disaster,disaster_type,disaster_severity,Resources/Help Type needed,location_mentioned,severity_score
0,0,True,Car Crash,Severe,Medical,Unknown,4
1,2,True,Earthquake,Mild,None,Different cities,3
2,3,True,Fire,Moderate,None,spot pond,3
3,9,True,Wildfire,Moderate,None,Spokane,3
4,11,True,Typhoon,Severe,Medical,China and Taiwan,5
...,...,...,...,...,...,...,...
1358,10861,True,Earthquake,Mild,Shelter,Los Angeles,2
1359,10865,True,Storm,Severe,Power,RI,4
1360,10868,True,Wreck,Moderate,Transportation,Chicago,3
1361,10874,True,Storm,Mild,Shelter,Unknown,2


In [63]:
#Storing every result in local memory so that we need not redo everything if by chance get stopped in between and call API again
#Firstly storing batch results text received from API as key value pair in the form of dataframe
API_results = pd.DataFrame.from_dict(batch_results, orient='index')
API_results.to_csv('Results/API_results.csv', index_label='batch_number')
API_results_text = pd.DataFrame(API_results[0].apply(lambda x: x.text))
API_results_text.to_csv('Results/API_results_text.csv', index_label='batch_number')
final_results_df.to_csv('Results/disaster_tweet_analysis_results.csv', index=False)

In [65]:
sub_batch_result = {}


In [ ]:
#Checking for all the texts processed i.e. 33 batches if a batch is not processed we will call for that batch only again
# Found problem those batches were not processed becuse due to rate limiting the complete JSON was so we will reprocess these non-processed batches in batches of 75-75 again combining 3 100 value non processed batches and then breaking them into 4 sub batches of 75 
#THen if we comnined we need to increment i by 3 to avoid reprocessing the same batches again
# We will store these sub batch results also in a separate file
process_received_text = batch_results
for i in range(num_batches):  
    batch_df = process_received_text_toDataFrame(batch_results[i].text)
    if (type(batch_df) == str) and (batch_df == "JSON Not Found"):
        for j in range(3):  # Combine 3 batches of 100 each
            combined_start = (i + j) * batch_size
            combined_end = min((i + j + 1) * batch_size, len(test_df))
            if combined_start >= len(test_df):
                break
            if j == 0:
                combined_tweets = test_df.iloc[combined_start:combined_end]
            else:
                combined_tweets = pd.concat([combined_tweets, test_df.iloc[combined_start:combined_end]], ignore_index=True)
        # Now split combined_tweets into 4 sub-batches of 75 each
        sub_batch_size = 75
        num_sub_batches = (len(combined_tweets) + sub_batch_size - 1) // sub_batch_size
        for k in range(num_sub_batches):
            sub_batch_start = k * sub_batch_size
            sub_batch_end = min((k + 1) * sub_batch_size, len(combined_tweets))
            sub_batch = combined_tweets.iloc[sub_batch_start:sub_batch_end]
            print(f"Reprocessing Sub-Batch {k+1} of Combined Batch {i+1}...")
            sub_batch_result[i] = send_tweets_in_batches(sub_batch)
            sub_batch_df = process_received_text_toDataFrame(sub_batch_result[i].text)
            #Need to first check if batch_df is not a dataframe then check for string
            if (type(sub_batch_df) == str) and (sub_batch_df == "JSON Not Found"):
                print(f"Sub-Batch {k+1} of Combined Batch {i+1} still failed to produce valid JSON. Skipping this sub-batch.")
                continue  # Skip this sub-batch if it still fails
            if not sub_batch_df.empty:
                final_results_df = pd.concat([final_results_df, sub_batch_df], ignore_index=True)
        i += 2  # Increment i by 2 more to skip the next two batches already combined
    else:
        print(f"Batch {i+1} was already processed successfully.")
        continue  # This batch was already processed successfully

In [70]:
# Store results again after reprocessing the sub batches in another file
API_results_sub = pd.DataFrame.from_dict(sub_batch_result, orient='index')
API_results_sub.to_csv('Results/API_results_sub.csv', index_label='sub_batch_number')
API_results_sub_text = pd.DataFrame(API_results_sub[0].apply(lambda x: x.text))
API_results_sub_text.to_csv('Results/API_results_sub_text.csv', index_label='sub_batch_number')
final_results_df.to_csv('Results/disaster_tweet_analysis_results_final.csv', index=False)
#!/usr/bin/env python3

In [87]:
temporary_results = final_results_df
temporary_results.to_csv('Results/temporary_results.csv', index=False)
temporary_results

,id,is_disaster,disaster_type,disaster_severity,Resources/Help Type needed,location_mentioned,severity_score
0,0,True,Car Crash,Severe,Medical,Unknown,4
1,2,True,Earthquake,Mild,None,Different cities,3
2,3,True,Fire,Moderate,None,spot pond,3
3,9,True,Wildfire,Moderate,None,Spokane,3
4,11,True,Typhoon,Severe,Medical,China and Taiwan,5
...,...,...,...,...,...,...,...
10583,10701,False,None,None,None,None,0
10584,10703,False,None,None,None,None,0
10585,10713,False,None,None,None,None,0
10586,10714,True,Wreck,Moderate,Medical,None,3


In [88]:
# Removing redundant values from temporary results on basis of id as unique identifier
temporary_results_unique = temporary_results.drop_duplicates(subset=['id'])
temporary_results_unique.to_csv('Results/temporary_results_unique.csv', index=False)
temporary_results_unique

,id,is_disaster,disaster_type,disaster_severity,Resources/Help Type needed,location_mentioned,severity_score
0,0,True,Car Crash,Severe,Medical,Unknown,4
1,2,True,Earthquake,Mild,None,Different cities,3
2,3,True,Fire,Moderate,None,spot pond,3
3,9,True,Wildfire,Moderate,None,Spokane,3
4,11,True,Typhoon,Severe,Medical,China and Taiwan,5
...,...,...,...,...,...,...,...
10358,10618,True,Shooting,Moderate,Medical,"Auburn Gresham, Chicago, Illinois",3
10359,10619,True,Attack,Moderate,Medical,Singapore,3
10360,10624,True,Shooting,Moderate,Medical,SWMO,3
10361,10633,True,Shooting,Moderate,Medical,Unknown,3


In [89]:
#Combining API results JSONs from bothe batches and sub batches and storing them also so that we can process them afeterwards for their JSON extraction also
combined_API_results = pd.concat([API_results, API_results_sub], ignore_index=True)
combined_API_results.to_csv('Results/combined_API_results.csv', index_label='batch_number')
combined_API_results_text = pd.DataFrame(combined_API_results[0].apply(lambda x: x.text))
combined_API_results_text.to_csv('Results/combined_API_results_text.csv', index_label='batch_number')
combined_API_results_text

,0
0,"```json\n[\n {\n ""id"": 0,\n ""is_disaste..."
1,"```json\n[\n {\n ""id"": 339,\n ""is_disas..."
2,"```json\n[\n {\n ""id"": 647,\n ""is_disas..."
3,"```json\n[\n {\n ""id"": 975,\n ""is_disas..."
4,"```json\n[\n {\n ""id"": 1295,\n ""is_disa..."
5,"```json\n[\n {\n ""id"": 1635,\n ""is_disa..."
6,"```json\n[\n {\n ""id"": 1947,\n ""is_disa..."
7,"```json\n[\n {\n ""id"": 2276,\n ""is_disa..."
8,"```json\n[\n {\n ""id"": 2630,\n ""is_disa..."
9,"```json\n[\n {\n ""id"": 2968,\n ""is_disa..."


In [102]:
#A single function to process all combined API results text and convert them to a single dataframe
combined_final_results_df = pd.DataFrame()
def process_all_combined_API_results(combined_API_results_text: pd.DataFrame):
    combined_final_results_df = pd.DataFrame()
    for i, row in combined_API_results_text.iterrows():
        batch_df = process_received_text_toDataFrame(row[0])
        if (type(batch_df) == str) and (batch_df == "JSON Not Found"):
            print(f"Batch {i+1} still failed to produce valid JSON. Skipping this batch.")
            continue  # Skip this batch if it fails
        if not batch_df.empty:
            # Option A: concat as you go
            combined_final_results_df = pd.concat(
                [combined_final_results_df, batch_df], ignore_index=True
            )
    return combined_final_results_df

In [103]:
final_combined_results_df = process_all_combined_API_results(combined_API_results_text)

--- Successfully Extracted JSON String ---
[
  {
    "id": 0,
    "is_disaster": true,
    "disaster_type": "Car Crash",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "Medical",
    "location_mentioned": "Unknown",
    "severity_score": 3
  },
  {
    "id": 2,
    "is_disaster": true,
    "disaster_type": "Earthquake",
    "disaster_severity": "Mild",
    "Resources/Help Type needed": "None",
    "location_mentioned": "Unknown",
    "severity_score": 2
  },
  {
    "id": 3,
    "is_disaster": true,
    "disaster_type": "Fire",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "Animal Rescue",
    "location_mentioned": "spot pond",
    "severity_score": 3
  },
  {
    "id": 9,
    "is_disaster": true,
    "disaster_type": "Wildfire",
    "disaster_severity": "Moderate",
    "Resources/Help Type needed": "None",
    "location_mentioned": "Spokane",
    "severity_score": 3
  },
  {
    "id": 11,
    "is_disaster": true,
    "disaster_type": "T

In [104]:
final_combined_results_df

,id,is_disaster,disaster_type,disaster_severity,Resources/Help Type needed,location_mentioned,severity_score
0,0,True,Car Crash,Moderate,Medical,Unknown,3
1,2,True,Earthquake,Mild,None,Unknown,2
2,3,True,Fire,Moderate,Animal Rescue,spot pond,3
3,9,True,Wildfire,Moderate,None,Spokane,3
4,11,True,Typhoon,Severe,"Medical, Shelter",China and Taiwan,5
...,...,...,...,...,...,...,...
2683,10701,False,None,None,None,None,0
2684,10703,False,None,None,None,None,0
2685,10713,False,None,None,None,None,0
2686,10714,True,Wreck,Moderate,Medical,None,3


In [106]:
#Since our temporary results already has all the results we will just save it as final
final_df = temporary_results_unique
final_df.to_csv('Results/final_disaster_tweet_analysis_results.csv', index=False)
final_df

,id,is_disaster,disaster_type,disaster_severity,Resources/Help Type needed,location_mentioned,severity_score
0,0,True,Car Crash,Severe,Medical,Unknown,4
1,2,True,Earthquake,Mild,None,Different cities,3
2,3,True,Fire,Moderate,None,spot pond,3
3,9,True,Wildfire,Moderate,None,Spokane,3
4,11,True,Typhoon,Severe,Medical,China and Taiwan,5
...,...,...,...,...,...,...,...
10358,10618,True,Shooting,Moderate,Medical,"Auburn Gresham, Chicago, Illinois",3
10359,10619,True,Attack,Moderate,Medical,Singapore,3
10360,10624,True,Shooting,Moderate,Medical,SWMO,3
10361,10633,True,Shooting,Moderate,Medical,Unknown,3


In [108]:
#Also saving results in asked Kaggle format to submit and increse ranking
kaggle_submission = final_df[['id','is_disaster']]
#Changing is_disaster column name to target as asked in Kaggle submission format
kaggle_submission = kaggle_submission.rename(columns={'is_disaster': 'target'})
kaggle_submission.to_csv('Results/kaggle_submission.csv', index=False)
kaggle_submission

,id,target
0,0,True
1,2,True
2,3,True
3,9,True
4,11,True
...,...,...
10358,10618,True
10359,10619,True
10360,10624,True
10361,10633,True


In [109]:
#For target variable we want 0 0r 1 so mapping accordingly
kaggle_submission['target'] = kaggle_submission['target'].map({True: 1, False: 0})
kaggle_submission.to_csv('Results/kaggle_submission.csv', index=False)
kaggle_submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
10358,10618,1
10359,10619,1
10360,10624,1
10361,10633,1
